## Apply an image-based profiling pipeline using pycytominer

As described fully in [Caicedo et al. 2017](https://doi.org/10.1038/nmeth.4397), an image-based profiling pipeline consists of three core steps:

1. Aggregation
2. Normalization
3. Feature selection

[Pycytominer](https://github.com/cytomining/pycytominer) is a python package, built on top of pandas, that facilitates all of these steps and more.

### Data levels

The concept of "data levels" is important to understand when implementing an image-based profiling pipeline.

| Data | Level |
| :---- | :---- |
| Images | Level 1 |
| Single cell profiles (SQLite) | Level 2 |
| Aggregated profiles with metadata information | Level 3 |
| Normalized aggregated profiles | Level 4a |
| Normalized and feature selected profiles | Level 4b |
| Consensus profiles | Level 5 |

In [1]:
import pathlib
import pandas as pd

from pycytominer.aggregate import AggregateProfiles
from pycytominer import annotate, normalize, feature_select, consensus

### Step 0 - Initialize data

In [2]:
data_dir = "data"
plate_id = "218360"
platemap = "218360.txt"

sqlite_file = f"sqlite:///{data_dir}/{plate_id}.sqlite"
sqlite_file

'sqlite:///data/218360.sqlite'

In [3]:
output_dir = pathlib.Path("profiles")
output_dir.mkdir(exist_ok=True)

In [4]:
# Load platemap file
platemap_file = pathlib.Path(f"{data_dir}/{platemap}")
platemap_df = pd.read_csv(platemap_file, sep="\t")

print(platemap_df.shape)
platemap_df.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'data/218360.txt'

### Step 1 - Single cell aggregation

In this step, **level 2 profiles** (single cells) are processed to **level 3 profiles** (well-level profiles).

In [5]:
# Initialize the aggregation step
ap = AggregateProfiles(
    sqlite_file,
    strata=["Metadata_Plate", "Metadata_Well"],
    features="infer",
    operation="median",
    output_file="none",
    compartments=["cells", "cytoplasm", "nuclei"],
    merge_cols=["TableNumber", "ImageNumber"],
    load_image_data=True,
    subsample_frac=1,
    subsample_n="all",
    subsampling_random_state="none"
)

In [6]:
# Count cells
cell_count_df = ap.count_cells()

cell_count_df = (
    cell_count_df.merge(
        platemap_df,
        left_on="Metadata_Well",
        right_on="well_position"
    )
)

cell_count_file = pathlib.Path(f"{output_dir}/cell_counts.tsv")
cell_count_df.to_csv(cell_count_file, sep="\t", index=False)

print(cell_count_df.shape)
cell_count_df.head()

(60, 11)


,Metadata_Plate,Metadata_Well,cell_count,plate_map_name,well_position,clone_number,plate_ID,plate_filename,treatment,treatment_time,cell_density
0,218360,B02,1386,218360,B02,WT parental,218360,20200626-WTpAE-Lo,0.1% DMSO,13 hr,2.5x10^3 cells/well
1,218360,B03,793,218360,B03,WT parental,218360,20200626-WTpAE-Lo,2.1 nM bortezomib,13 hr,2.5x10^3 cells/well
2,218360,B04,456,218360,B04,WT parental,218360,20200626-WTpAE-Lo,21 nM bortezomib,13 hr,2.5x10^3 cells/well
3,218360,B05,258,218360,B05,WT parental,218360,20200626-WTpAE-Lo,210 nM bortezomib,13 hr,2.5x10^3 cells/well
4,218360,B06,1094,218360,B06,WT parental,218360,20200626-WTpAE-Lo,0.1% DMSO,13 hr,2.5x10^3 cells/well


In [7]:
# Perform the aggregation - output well level profiles
output_file = pathlib.Path(f"{output_dir}/{plate_id}.csv.gz")

# Aggregate profiles can output a file, or save the result to a variable
# Here, we output the intermediate result to a file
ap.aggregate_profiles(
    output_file=output_file,
    compute_subsample=True,
    compression="gzip",
    float_format=None
)

In [8]:
# Read in and preview what was output in the previous step 
aggregated_df = pd.read_csv(output_file)

print(aggregated_df.shape)
aggregated_df.head(3)

(60, 3530)


,Metadata_Plate,Metadata_Well,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Center_Z,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,...,Nuclei_Texture_Variance_RNA_10_02,Nuclei_Texture_Variance_RNA_10_03,Nuclei_Texture_Variance_RNA_20_00,Nuclei_Texture_Variance_RNA_20_01,Nuclei_Texture_Variance_RNA_20_02,Nuclei_Texture_Variance_RNA_20_03,Nuclei_Texture_Variance_RNA_5_00,Nuclei_Texture_Variance_RNA_5_01,Nuclei_Texture_Variance_RNA_5_02,Nuclei_Texture_Variance_RNA_5_03
0,218360,B02,3103.0,1185.5,1134.5,1.0,1.176543,0.762628,1.0,0.584687,...,134.630929,136.168955,130.178186,143.230289,130.452067,139.230146,135.932245,133.168297,137.391985,135.095233
1,218360,B03,3365.0,1067.0,1137.0,1.0,1.148519,0.738471,1.0,0.605072,...,143.936790,150.720200,155.195390,182.867025,153.439750,162.839254,146.040203,143.993466,147.316124,147.909953
2,218360,B04,3360.5,1141.0,1166.5,1.0,1.152265,0.735413,1.0,0.608022,...,139.837458,147.532470,143.564510,152.944447,143.430507,147.967005,140.323478,141.193128,140.788792,140.561236


## Step 2 - Annotate wells using the platemap file

In this step, **level 3 profiles** (well-level profiles) are annotated with platemap metadata.

In [9]:
# Annotate profiles
annotate_file = pathlib.Path(f"{output_dir}/{plate_id}_augmented.csv.gz")

annotate(
    profiles=output_file,
    platemap=platemap_df,
    join_on=["Metadata_well_position", "Metadata_Well"],
    output_file=annotate_file,
    compression="gzip"
)

In [10]:
# Read in and preview what was output in the previous step 
annotated_df = pd.read_csv(annotate_file)

print(annotated_df.shape)
annotated_df.head(3)

(60, 3537)


,Metadata_plate_map_name,Metadata_clone_number,Metadata_plate_ID,Metadata_plate_filename,Metadata_treatment,Metadata_treatment_time,Metadata_cell_density,Metadata_Plate,Metadata_Well,Cells_AreaShape_Area,...,Nuclei_Texture_Variance_RNA_10_02,Nuclei_Texture_Variance_RNA_10_03,Nuclei_Texture_Variance_RNA_20_00,Nuclei_Texture_Variance_RNA_20_01,Nuclei_Texture_Variance_RNA_20_02,Nuclei_Texture_Variance_RNA_20_03,Nuclei_Texture_Variance_RNA_5_00,Nuclei_Texture_Variance_RNA_5_01,Nuclei_Texture_Variance_RNA_5_02,Nuclei_Texture_Variance_RNA_5_03
0,218360,WT parental,218360,20200626-WTpAE-Lo,0.1% DMSO,13 hr,2.5x10^3 cells/well,218360,B02,3103.0,...,134.630929,136.168955,130.178186,143.230289,130.452067,139.230146,135.932245,133.168297,137.391985,135.095233
1,218360,WT parental,218360,20200626-WTpAE-Lo,2.1 nM bortezomib,13 hr,2.5x10^3 cells/well,218360,B03,3365.0,...,143.936790,150.720200,155.195390,182.867025,153.439750,162.839254,146.040203,143.993466,147.316124,147.909953
2,218360,WT parental,218360,20200626-WTpAE-Lo,21 nM bortezomib,13 hr,2.5x10^3 cells/well,218360,B04,3360.5,...,139.837458,147.532470,143.564510,152.944447,143.430507,147.967005,140.323478,141.193128,140.788792,140.561236


## Step 3 - Normalize well-level profiles

In this step, **level 3 profiles** (well-level profiles) are normalized to form **level 4a profiles**.

In [11]:
# Normalize profiles
normalize_file = pathlib.Path(f"{output_dir}/{plate_id}_normalized.csv.gz")

normalize(
    profiles=annotate_file,
    features="infer",
    meta_features="infer",
    samples="Metadata_treatment == '0.1% DMSO'",
    method="standardize",
    output_file=normalize_file,
    compression="gzip"
)

In [12]:
# Read in and preview what was output in the previous step 
normalized_df = pd.read_csv(normalize_file)

print(normalized_df.shape)
normalized_df.head(3)

(60, 3537)


,Metadata_plate_map_name,Metadata_clone_number,Metadata_plate_ID,Metadata_plate_filename,Metadata_treatment,Metadata_treatment_time,Metadata_cell_density,Metadata_Plate,Metadata_Well,Cells_AreaShape_Area,...,Nuclei_Texture_Variance_RNA_10_02,Nuclei_Texture_Variance_RNA_10_03,Nuclei_Texture_Variance_RNA_20_00,Nuclei_Texture_Variance_RNA_20_01,Nuclei_Texture_Variance_RNA_20_02,Nuclei_Texture_Variance_RNA_20_03,Nuclei_Texture_Variance_RNA_5_00,Nuclei_Texture_Variance_RNA_5_01,Nuclei_Texture_Variance_RNA_5_02,Nuclei_Texture_Variance_RNA_5_03
0,218360,WT parental,218360,20200626-WTpAE-Lo,0.1% DMSO,13 hr,2.5x10^3 cells/well,218360,B02,0.951652,...,1.994853,1.918636,2.082421,1.807392,2.111036,2.086668,1.950899,1.943007,1.988028,1.987186
1,218360,WT parental,218360,20200626-WTpAE-Lo,2.1 nM bortezomib,13 hr,2.5x10^3 cells/well,218360,B03,2.645488,...,2.343617,2.448316,3.071418,3.091680,3.011963,2.951846,2.326277,2.351921,2.354838,2.466231
2,218360,WT parental,218360,20200626-WTpAE-Lo,21 nM bortezomib,13 hr,2.5x10^3 cells/well,218360,B04,2.616395,...,2.189983,2.332280,2.611618,2.122145,2.619684,2.406839,2.113976,2.246140,2.113579,2.191518


## Step 4 - Apply feature selection

In this step, we apply a series of feature selection steps to **level 4a profiles** (normalized well-level profiles) to form **level 4b profiles**.

In [13]:
# Apply feature selection
feature_select_file = pathlib.Path(f"{output_dir}/{plate_id}_normalized_feature_select.csv.gz")

feature_select_opts = [
    "variance_threshold",
    "drop_na_columns",
    "correlation_threshold",
    "blocklist",
    "drop_outliers"
]

feature_select(
    profiles=normalize_file,
    features="infer",
    samples="all",
    operation=feature_select_opts,
    output_file=feature_select_file,
    compression="gzip",
)

In [14]:
# Read in and preview what was output in the previous step 
feature_select_df = pd.read_csv(feature_select_file)

print(feature_select_df.shape)
feature_select_df.head(3)

(60, 389)


,Metadata_plate_map_name,Metadata_clone_number,Metadata_plate_ID,Metadata_plate_filename,Metadata_treatment,Metadata_treatment_time,Metadata_cell_density,Metadata_Plate,Metadata_Well,Cells_AreaShape_FormFactor,...,Nuclei_Texture_InfoMeas2_ER_5_00,Nuclei_Texture_InfoMeas2_Mito_20_03,Nuclei_Texture_InfoMeas2_RNA_5_02,Nuclei_Texture_InverseDifferenceMoment_AGP_20_01,Nuclei_Texture_InverseDifferenceMoment_DNA_20_03,Nuclei_Texture_InverseDifferenceMoment_ER_20_03,Nuclei_Texture_InverseDifferenceMoment_Mito_20_03,Nuclei_Texture_InverseDifferenceMoment_RNA_20_00,Nuclei_Texture_SumVariance_ER_5_01,Nuclei_Texture_SumVariance_RNA_10_02
0,218360,WT parental,218360,20200626-WTpAE-Lo,0.1% DMSO,13 hr,2.5x10^3 cells/well,218360,B02,-2.215826,...,1.559663,0.666380,1.640270,-1.123220,-0.511368,-1.301264,-0.172448,-1.220708,1.750259,1.982705
1,218360,WT parental,218360,20200626-WTpAE-Lo,2.1 nM bortezomib,13 hr,2.5x10^3 cells/well,218360,B03,-2.662273,...,1.341650,1.022193,0.938770,-0.321421,-1.862488,-0.175040,0.309052,-0.832363,2.854113,2.137837
2,218360,WT parental,218360,20200626-WTpAE-Lo,21 nM bortezomib,13 hr,2.5x10^3 cells/well,218360,B04,0.821753,...,0.164014,1.378143,0.603039,-0.322174,-2.405300,0.392906,-1.128065,-0.716326,1.869677,1.554196


## Step 5 - Form consensus signatures

In this step, we collapse replicates (**level 4 profiles**) into a single profile.
This forms **level 5 profiles**.

In [15]:
# Generate consensus profiles
consensus_file = pathlib.Path(f"{output_dir}/{plate_id}_consensus.csv.gz")

consensus(
    profiles=feature_select_df,
    replicate_columns=["Metadata_clone_number", "Metadata_treatment"],
    operation="modz",
    features="infer",
    output_file=consensus_file,
    modz_method="spearman",
    modz_min_weight=0.01,
    modz_precision=4,
    compression="gzip",
)

In [16]:
# Read in and preview what was output in the previous step 
consensus_df = pd.read_csv(consensus_file)

print(consensus_df.shape)
consensus_df.head(3)

(12, 382)


,Metadata_clone_number,Metadata_treatment,Cells_AreaShape_FormFactor,Cells_AreaShape_Orientation,Cells_AreaShape_Zernike_0_0,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_3_3,Cells_AreaShape_Zernike_4_0,...,Nuclei_Texture_InfoMeas2_ER_5_00,Nuclei_Texture_InfoMeas2_Mito_20_03,Nuclei_Texture_InfoMeas2_RNA_5_02,Nuclei_Texture_InverseDifferenceMoment_AGP_20_01,Nuclei_Texture_InverseDifferenceMoment_DNA_20_03,Nuclei_Texture_InverseDifferenceMoment_ER_20_03,Nuclei_Texture_InverseDifferenceMoment_Mito_20_03,Nuclei_Texture_InverseDifferenceMoment_RNA_20_00,Nuclei_Texture_SumVariance_ER_5_01,Nuclei_Texture_SumVariance_RNA_10_02
0,Clone A,0.1% DMSO,-0.011786,0.039957,-0.944377,-0.569202,-0.722571,0.451732,0.670634,0.162143,...,-0.346070,-0.021621,-0.540451,1.043457,0.714507,0.684376,0.396256,0.833194,-0.631314,-0.628669
1,Clone A,2.1 nM bortezomib,-1.796962,-0.281173,-1.000878,-0.745920,-0.357371,0.240513,-0.810657,0.549817,...,-0.398871,0.471051,-0.615307,0.593170,-0.400998,0.166443,-0.364822,0.488448,-0.067216,-0.597910
2,Clone A,21 nM bortezomib,-3.364075,-0.129924,-1.831994,-0.116718,0.443118,1.053584,-3.051191,1.912930,...,0.655914,1.247328,-0.766947,0.969300,-1.974408,0.059174,-1.449950,0.349494,2.163107,-0.076069
